In [14]:
import os
import numpy as np
import pandas as pd
from sktime.datasets import load_from_tsfile_to_dataframe
from collections import defaultdict

In [15]:
path_to_datasets = "../datasets"
datasets_names = list()
datasets_paths = list()

# Print datasets' sizes

In [16]:
for name in os.listdir(path_to_datasets):
    print(name)
    datasets_names.append(name)
    datasets_paths.append(os.path.join(path_to_datasets, name))
    for path, dirs, files in os.walk(os.path.join("../datasets", name)):
        size = 0
        for f in files:
            fp = os.path.join(path, f)
            size += os.path.getsize(fp)
        print(size  >> 20, "Mb")

abnormal_heartbeat
85 Mb
arrow_head
0 Mb
coffee_beans
0 Mb
freezers
9 Mb
pedestrian_count_chinatown
0 Mb
right_whale_call
453 Mb
species_of_fruit_fly
1981 Mb
spoken_arabic_digits
35 Mb
tiselac
121 Mb


# Print num of test, train samples and classes' distribution

In [17]:
columns = ["ds_name", "length",  "train_size", "test_size", "train_classes_distr", "test_classes_distr"]
data = list()

In [18]:
def prepare_train_data(df):
    prepared = list()
    for j in range(df.size):
        prepared.append(df.iloc[j].to_numpy()[0].to_numpy())
    return np.array(prepared, dtype=np.float64)

def load_numpy_array_from_ts(path_to_file):
    x, y = load_from_tsfile_to_dataframe(path_to_file)
    return prepare_train_data(x), np.array(y, dtype=np.float64)

def array_dist(arr):
    len_arr = len(arr)
    res = defaultdict(int)
    for v in arr:
        res[v] += 1
    for k in res.keys():
        res[k] /= len_arr
    return res

In [22]:
for i in range(len(datasets_paths)):
    ds_path = datasets_paths[i]
    row = list()
    row.append(datasets_names[i])
    print(datasets_names[i])
    for file_name in os.listdir(ds_path):
        if "TEST" in file_name:
            test_file = os.path.join(ds_path, file_name)
        if "TRAIN" in file_name:
            train_file = os.path.join(ds_path, file_name)
    print(test_file, train_file)
    x_test, y_test = load_numpy_array_from_ts(test_file)
    x_train, y_train = load_numpy_array_from_ts(train_file)
    row.append(x_train.shape[1])
    row.append(x_train.shape[0])
    row.append(x_test.shape[0])
    row.append(str(dict(array_dist(y_train))))
    row.append(str(dict(array_dist(y_test))))
    data.append(row)

tiselac
../datasets\tiselac\Tiselac_TEST.ts ../datasets\tiselac\Tiselac_TRAIN.ts


IndexError: single positional indexer is out-of-bounds

In [24]:
dict(array_dist(y_test))

{0.0: 0.17567645865925025, 1.0: 0.29381771829190567, 2.0: 0.5305058230488441}

In [20]:
df = pd.DataFrame(data, columns=columns)

In [21]:
df

,ds_name,length,train_size,test_size,train_classes_distr,test_classes_distr
0,abnormal_heartbeat,18530,204,205,"defaultdict(<class 'int'>, {0.0: 0.73039215686...","defaultdict(<class 'int'>, {0.0: 0.73170731707..."
1,arrow_head,251,36,175,"defaultdict(<class 'int'>, {0.0: 0.33333333333...","defaultdict(<class 'int'>, {0.0: 0.39428571428..."
2,coffee_beans,286,28,28,"defaultdict(<class 'int'>, {0.0: 0.5, 1.0: 0.5})","defaultdict(<class 'int'>, {0.0: 0.53571428571..."
3,freezers,301,150,2850,"defaultdict(<class 'int'>, {1.0: 0.5, 2.0: 0.5})","defaultdict(<class 'int'>, {1.0: 0.5, 2.0: 0.5})"
4,pedestrian_count_chinatown,24,20,343,"defaultdict(<class 'int'>, {1.0: 0.5, 2.0: 0.5})","defaultdict(<class 'int'>, {1.0: 0.27405247813..."
5,right_whale_call,4000,10934,1962,"defaultdict(<class 'int'>, {1.0: 0.5, 0.0: 0.5})","defaultdict(<class 'int'>, {1.0: 0.5, 0.0: 0.5})"
6,species_of_fruit_fly,5000,17259,17259,"defaultdict(<class 'int'>, {0.0: 0.17567645865...","defaultdict(<class 'int'>, {0.0: 0.17567645865..."


In [27]:
df.to_csv("./datasets_analysis.csv", index=False)

# Notes - to delete